In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

**FORECAST METRICS**

In [2]:
def calculate_forecast_metrics(actual, predicted, model_name='Model'):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_sqaured_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted)/actual))*100

    #Additional Metrics
    bias = np.mean(predicted-actual)
    tracking_signal = np.sum(predicted-actual)/np.sum(np.abs(predicted-actual))

    return {
        'Model': model_name,
        'MAE': round(mae,2),
        'RMSE': round(rmse,2),
        'MAPE (%)': round(mape,2),
        'Bias': round(bias,2),
        'Tracking_signal': round(tracking_signal, 4)
    }

**CALCULATING SAFETY STOCK**

In [3]:
def calculate_safety_stock(forecast_error_std, lead_time_days, service_level=0.95):

    #Z-scores for common service levels
    z_scores = {
        0.90: 1.28,
        0.95: 1.65,
        0.99: 2.33,
        0.995: 2.58
    }
    z = z_scores.get(service_level,1.65) #default to 90%
    safety_stock = z * forecast_error_std * np.sqrt(lead_time_days)

    return round(safety_stock, 0)

**CALCULATING REORDER POINT**

In [4]:
def calculate_reorder_point(avg_daily_demand, lead_time_days, safety_stock):
    
    reorder_point = (avg_daily_demand * lead_time_days) + safety_stock
    return round (reorder_point,0)

**ECONOMIC ORDER QUANTITY**

In [5]:
def calculate_economic_order_quantity(annual_demand, ordering_cost, holding_cost_per_unit):
    eoq = np.sqrt((2*annual_demand * ordering_cost)/holding_cost_per_unit)
    return round(eoq,0)

**DETECTING OUTLIERS**

In [6]:
def detect_outliers_iqr(data, column, multiplier=1.5):
    
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

In [7]:
def create_lag_features(df, column, group_cols=None, lags=[1, 7, 14, 30]):

    df = df.copy()
    
    if group_cols:
        for lag in lags:
            df[f'{column}_lag_{lag}'] = df.groupby(group_cols)[column].shift(lag)
    else:
        for lag in lags:
            df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    
    return df

In [8]:
def create_rolling_features(df, column, group_cols=None, windows=[7, 14, 30]):
    
    df = df.copy()
    
    for window in windows:
        if group_cols:
            df[f'{column}_rolling_mean_{window}'] = df.groupby(group_cols)[column].transform(
                lambda x: x.rolling(window=window, min_periods=1).mean()
            )
            df[f'{column}_rolling_std_{window}'] = df.groupby(group_cols)[column].transform(
                lambda x: x.rolling(window=window, min_periods=1).std()
            )
        else:
            df[f'{column}_rolling_mean_{window}'] = df[column].rolling(
                window=window, min_periods=1
            ).mean()
            df[f'{column}_rolling_std_{window}'] = df[column].rolling(
                window=window, min_periods=1
            ).std()
    
    return df

In [21]:
def plot_forecast_comparison(actual, predictions_dict, title='Forecast Comparison', 
                            figsize=(14, 6), save_path=None):
    plt.figure(figsize=figsize)
    
    # Plot actual
    plt.plot(actual.index, actual.values, label='Actual', linewidth=2.5, color='black', marker='o', markersize=3)
    
    # Plot predictions
    colors = plt.cm.tab10(np.linspace(0, 1, len(predictions_dict)))
    for idx, (model_name, predictions) in enumerate(predictions_dict.items()):
        plt.plot(predictions.index, predictions.values, 
                label=model_name, linewidth=1.5, alpha=0.8, color=colors[idx])
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Sales Quantity', fontsize=12)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

In [22]:
def generate_sap_integration_report(forecast_data, material_id, plant, 
                                    lead_time, service_level=0.95):
    avg_demand = forecast_data['Sales_Quantity'].mean()
    forecast_error_std = forecast_data['Forecast_Error'].std()
    
    # Calculate parameters
    safety_stock = calculate_safety_stock(forecast_error_std, lead_time, service_level)
    reorder_point = calculate_reorder_point(avg_demand, lead_time, safety_stock)
    
    # 30-day forecast
    total_30day_forecast = forecast_data['Forecast'].tail(30).sum()
    
    report = {
        'Material_ID': material_id,
        'Plant': plant,
        'Avg_Daily_Demand': round(avg_demand, 0),
        'Lead_Time_Days': lead_time,
        'Service_Level': f"{service_level*100}%",
        'Safety_Stock': safety_stock,
        'Reorder_Point': reorder_point,
        '30_Day_Forecast': round(total_30day_forecast, 0),
        'Recommended_Order_Qty': round(total_30day_forecast + safety_stock, 0)
    }
    
    return report

In [24]:
def export_to_sap_format(df, output_path='sap_forecast_upload.csv'):
    
    # SAP IBP format typically requires: Material, Plant, Date, Forecast Quantity
    sap_df = df[['Material_ID', 'Plant', 'Date', 'Forecast_Quantity']].copy()
    sap_df.columns = ['MATERIAL', 'PLANT', 'CALDAY', 'FORECAST_QTY']
    
    # Format date as YYYYMMDD
    sap_df['CALDAY'] = pd.to_datetime(sap_df['CALDAY']).dt.strftime('%Y%m%d')
    
    # Export
    sap_df.to_csv(output_path, index=False)
    
    print(f"✅ SAP-formatted file exported: {output_path}")
    print(f"   Records: {len(sap_df):,}")
    
    return output_path

In [25]:
def calculate_inventory_metrics(current_inventory, forecast_demand, lead_time, 
                                holding_cost_per_unit_per_year):
    # Days of supply
    daily_demand = forecast_demand / 365
    days_of_supply = current_inventory / daily_demand if daily_demand > 0 else 0
    
    # Inventory turns
    inventory_turns = forecast_demand / current_inventory if current_inventory > 0 else 0
    
    # Holding cost
    annual_holding_cost = current_inventory * holding_cost_per_unit_per_year
    
    # Coverage ratio
    coverage_ratio = days_of_supply / lead_time if lead_time > 0 else 0
    
    return {
        'Days_of_Supply': round(days_of_supply, 1),
        'Inventory_Turns': round(inventory_turns, 2),
        'Annual_Holding_Cost': round(annual_holding_cost, 2),
        'Coverage_Ratio': round(coverage_ratio, 2)
    }

In [14]:
if __name__ == "__main__":
    print("Demand Forecasting Utility Functions")
    print("Available functions:")
    print("  - calculate_forecast_metrics()")
    print("  - calculate_safety_stock()")
    print("  - calculate_reorder_point()")
    print("  - calculate_economic_order_quantity()")
    print("  - detect_outliers_iqr()")
    print("  - create_time_features()")
    print("  - create_lag_features()")
    print("  - create_rolling_features()")
    print("  - plot_forecast_comparison()")
    print("  - generate_sap_integration_report()")
    print("  - export_to_sap_format()")
    print("  - calculate_inventory_metrics()")

Demand Forecasting Utility Functions
Available functions:
  - calculate_forecast_metrics()
  - calculate_safety_stock()
  - calculate_reorder_point()
  - calculate_economic_order_quantity()
  - detect_outliers_iqr()
  - create_time_features()
  - create_lag_features()
  - create_rolling_features()
  - plot_forecast_comparison()
  - generate_sap_integration_report()
  - export_to_sap_format()
  - calculate_inventory_metrics()
